In [1]:
import numpy as np
import pandas as pd
import math 
import itertools
import sklearn.impute
import tensorflow.keras

In [2]:
from sklearn import impute
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score


In [3]:
# Regression
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 

# Classification
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score


# Deep Learning Libraries
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.utils import to_categorical

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

In [5]:
from surprise import Reader, Dataset, KNNBasic, SVD, NMF
from surprise.model_selection import GridSearchCV, cross_validate

In [11]:
userId_and_productId=pd.read_csv('userId_and_product_for_20k.csv')

In [12]:
userId_and_productId=userId_and_productId.drop(["Unnamed: 0"],axis=1)


In [13]:
userId_and_productId

,userId,movieId
0,576802932,5712790
1,576802932,5650294
2,576802932,5692261
3,576802932,5694628
4,576802932,5832416
...,...,...
211745,539393202,5700037
211746,581084906,5859450
211747,581084906,5910231
211748,581084906,5830562


In [14]:
ratings=pd.read_csv('ratings_for_20k.csv')

In [15]:
ratings

,Unnamed: 0,rating
0,0,2.004808
1,1,2.004808
2,2,2.004808
3,3,2.004808
4,4,0.004808
...,...,...
211745,211745,3.166667
211746,211746,0.250000
211747,211747,0.250000
211748,211748,0.250000


In [16]:
ratings=ratings.drop(["Unnamed: 0"],axis=1)

In [17]:
ratings

,rating
0,2.004808
1,2.004808
2,2.004808
3,2.004808
4,0.004808
...,...
211745,3.166667
211746,0.250000
211747,0.250000
211748,0.250000


In [21]:
data_all=userId_and_productId.join(other=ratings)
data_all

,userId,movieId,rating
0,576802932,5712790,2.004808
1,576802932,5650294,2.004808
2,576802932,5692261,2.004808
3,576802932,5694628,2.004808
4,576802932,5832416,0.004808
...,...,...,...
211745,539393202,5700037,3.166667
211746,581084906,5859450,0.250000
211747,581084906,5910231,0.250000
211748,581084906,5830562,0.250000


In [22]:
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df( data_all[['userId', 'movieId', 'rating']], reader = reader )

In [23]:
sim_options = {'name' : 'msd'}

algo = KNNBasic(k=20, sim_options=sim_options )
cross_validate(algo=algo, data=data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3822  1.3895  1.3828  1.3896  1.3889  1.3866  0.0034  
Fit time          5.46    5.44    5.11    5.40    5.17    5.32    0.15    
Test time         1.10    1.07    1.08    1.19    1.17    1.12    0.05    


{'test_rmse': array([1.38215523, 1.38947037, 1.38275261, 1.38962255, 1.38893333]),
 'fit_time': (5.45784854888916,
  5.440212249755859,
  5.110471725463867,
  5.397968053817749,
  5.168620347976685),
 'test_time': (1.0970714092254639,
  1.0695412158966064,
  1.0770504474639893,
  1.194983959197998,
  1.168975830078125)}

In [24]:
n_neighbours = [3,5,7,11,15]
param_grid = {'n_neighbours' : n_neighbours}

gs = GridSearchCV(KNNBasic, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

print('\n\n###############')
# Best RMSE score
print('Best Score :', gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print('Best Parameters :', gs.best_params['rmse'])
print('###############')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [25]:
algo = SVD()
cross_validate(algo=algo, data=data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1422  1.1375  1.1382  1.1374  1.1428  1.1396  0.0023  
Fit time          6.59    6.76    6.70    6.71    6.76    6.71    0.06    
Test time         0.20    0.29    0.29    0.20    0.19    0.23    0.04    


{'test_rmse': array([1.14217752, 1.13754609, 1.13817621, 1.13743006, 1.1427545 ]),
 'fit_time': (6.594798564910889,
  6.76289439201355,
  6.704494476318359,
  6.709139585494995,
  6.758752107620239),
 'test_time': (0.19839978218078613,
  0.2877993583679199,
  0.2872014045715332,
  0.2044525146484375,
  0.18650031089782715)}

In [26]:
param_grid = {'n_factors' : [50, 75, 100], 'lr_all' : [0.5, 0.05, 0.001], 'reg_all' : [0.06, 0.04, 0.02]}

gs = GridSearchCV(algo_class=SVD, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

print('\n###############')
# Best RMSE score
print('Best Score :', gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print('Best Parameters :', gs.best_params['rmse'])
print('###############')


###############
Best Score : 1.14022183373585
Best Parameters : {'n_factors': 100, 'lr_all': 0.05, 'reg_all': 0.06}
###############
